<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/Hugo_Abonizio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Hugo Abonizio'
print(f'Meu nome é {nome}')

Meu nome é Hugo Abonizio


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.6 MB/s 
     |████████████████████████████████| 596 kB 35.8 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 6.6 MB 45.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 13:44:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List
from tqdm.auto import tqdm


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


# class MyDataset():
#     def __init__(self, texts: List[str], tokenizer, context_size: int):
#         self.examples = []
#         for text in tqdm_notebook(texts):
#             token_ids = tokenize(text=text, tokenizer=tokenizer)
#             if len(token_ids) < context_size + 1:
#                 continue
#             # Compute n-grams:
#             for i in range(len(token_ids) - context_size):
#                 input_ids = token_ids[i:i + context_size]
#                 target_id = token_ids[i + context_size]
#                 self.examples.append((input_ids, target_id)) 

#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, idx):
#         input_ids, target_id = self.examples[idx]
#         return torch.LongTensor(input_ids), target_id

class MyDataset:
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.examples = []
        for text in tqdm(texts):
          tokens = tokenize(text, tokenizer)
          contexts = [tokens[i:(i + context_size + 1)] for i in range(len(tokens) - context_size)]
          self.examples += contexts

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        *inputs, target = self.examples[idx]
        return (torch.tensor(inputs), torch.tensor(target))

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-25 13:45:06--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.128, 74.125.197.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   296MB/s    in 0.4s    

2022-05-25 13:45:07 (296 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [9]:
import os
import gc
gc.collect()
import pickle

# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

# print('Truncating for debugging purposes.')
# texts = texts[:500]

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

if os.path.exists('saved-dataset.pickle'):
    with open('saved-dataset.pickle', 'rb') as f:
      training_dataset = pickle.load(f)
else:
  training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
  with open('saved-dataset.pickle', 'wb') as f:
      pickle.dump(training_dataset, f)

# training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

  0%|          | 0/24800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
import gc
gc.collect()

0

In [11]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


In [87]:
import torch.nn.functional as F

class LanguageModel(torch.nn.Module):
    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        super().__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.positional_embeddings = nn.Embedding(context_size, embedding_dim)

        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.W_o = nn.Linear(embedding_dim, embedding_dim, bias=False)

        hidden_size = embedding_dim

        self.feed_forward = nn.Sequential(
            torch.nn.Linear(embedding_dim, hidden_size),
            torch.nn.ReLU(),
            # a LM head do GPT-2 não tem bias
            # https://github.com/huggingface/transformers/blob/4d727bd2dff377caeab21ff4e1bf4b26c2397c8a/src/transformers/models/gpt2/modeling_gpt2.py#L953
            # torch.nn.Linear(hidden_size, vocab_size, bias=False),
            torch.nn.Linear(hidden_size, vocab_size),
        )


    def forward(self, inputs):
    #     """
    #     Args:
    #         inputs is a LongTensor of shape (batch_size, context_size)
            
    #     Returns:
    #         logits of shape (batch_size, vocab_size)
    #     """
        x = self.embeddings(inputs) # + self.positional_embeddings.weight
        pos_ids = torch.arange(inputs.size(1), dtype=torch.long, device=device)
        pos_embeddings = self.positional_embeddings(pos_ids)
        x += pos_embeddings
        
        residual = x[:, -1, :]

        q = self.W_q(residual.unsqueeze(dim=1)) # último token
        k = self.W_k(x)
        v = self.W_v(x)

        scores = torch.matmul(q, k.transpose(2, 1))
        probs = F.softmax(scores, dim=-1)
        e = torch.matmul(probs, v)
        out = e.squeeze(dim=1)

        out = out.squeeze(dim=1)
        out = self.W_o(out)

        out += residual

        out = self.feed_forward(out)
        return out

## Teste o modelo com um exemplo

In [95]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
# model(sample_train_gpu)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [89]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 7740258


## Assert da Perplexidade


In [90]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              32307
correct initial perplexity: 29794
Passou o no assert da perplexidade


In [91]:
import gc
model = None
gc.collect()

375

## Laço de Treinamento e Validação

In [102]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

max_examples = 400_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
best_valid_ppl = float('inf')
n_examples = 0
step = 0
while n_examples < max_examples:
    gc.collect()
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))
                if valid_ppl < best_valid_ppl:
                  best_valid_ppl = valid_ppl
                  torch.save(model, 'best_model.pt')

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 33040.08, valid ppl: 31853.69
10000 steps; 10240000 examples so far; train ppl: 649.96, valid ppl: 399.21
20000 steps; 20480000 examples so far; train ppl: 343.87, valid ppl: 308.73
30000 steps; 30720000 examples so far; train ppl: 289.84, valid ppl: 279.29
40000 steps; 40960000 examples so far; train ppl: 264.84, valid ppl: 263.64
50000 steps; 51200000 examples so far; train ppl: 254.08, valid ppl: 252.74
60000 steps; 61440000 examples so far; train ppl: 242.80, valid ppl: 247.98
70000 steps; 71680000 examples so far; train ppl: 236.77, valid ppl: 243.05
80000 steps; 81920000 examples so far; train ppl: 234.21, valid ppl: 239.30
90000 steps; 92160000 examples so far; train ppl: 225.98, valid ppl: 237.13
100000 steps; 102400000 examples so far; train ppl: 225.58, valid ppl: 233.91
110000 steps; 112640000 examples so far; train ppl: 223.48, valid ppl: 232.74
120000 steps; 122880000 examples so far; train ppl: 218.31, valid ppl: 231.19
130000 steps;

KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
best_model = torch.load('best_model.pt')

In [104]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 206.47739134917876


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [105]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz parte
Eu gosto de comer pizza pois me faz parte da
Eu gosto de comer pizza pois me faz parte da minha
Eu gosto de comer pizza pois me faz parte da minhas
Eu gosto de comer pizza pois me faz parte da minhas,
Eu gosto de comer pizza pois me faz parte da minhas, que
Eu gosto de comer pizza pois me faz parte da minhas, que a
Eu gosto de comer pizza pois me faz parte da minhas, que a sua
Eu gosto de comer pizza pois me faz parte da minhas, que a sua vez
Eu gosto de comer pizza pois me faz parte da minhas, que a sua vez mais
Eu gosto de comer pizza pois me faz parte da minhas, que a sua vez mais de
Eu gosto de comer pizza pois me faz parte da minhas, que a sua vez mais de um
Eu gosto de comer pizza pois me faz parte da minhas, que a sua vez mais de um dos
Eu gosto de comer pizza pois me faz parte da minhas, que a sua vez mais de um dos seus
Eu gosto de comer pizza pois me faz parte da minhas, que a sua vez mais de um dos seus próprios
Eu gosto de comer piz